In [1]:
import re
import numpy as np

with open('../Input/a7.txt') as input_file:
    bags = input_file.read().split('\n')

In [2]:
for bag in bags:
    main_bag    = re.sub('^(.*?) bags.*?$', '\\1', bag)
    inside_bags = re.sub('^.*? bags contain (.*?)\\.$', '\\1', bag)#.split(', ')
    try :
        bags_map = np.vstack([bags_map, np.array([main_bag, inside_bags])])
    except:
        bags_map = np.array([main_bag, inside_bags])

In [3]:
################################################################
## PART 1 | Could be re-written as recursive function
################################################################
def find_outer_bags(look_for, bags):
    checked_bags = []
    while True:
        checked_bags = np.append(checked_bags, look_for)
        look_for = '|'.join(set(look_for))
        mask = [bool(re.search(look_for, bag)) for bag in  bags[:, 1]]

        look_for = np.setdiff1d(bags[mask, ][:, 0], checked_bags)
        if len(bags[mask, ]) == 0 or all(look_for == ''):
            break

        try:
            bag_check = np.c_[bag_check, mask]  
        except:
            bag_check = mask

    count_outer_bags = sum(bag_check.sum(axis = 1) != 0)
    outer_bags       = bags[bag_check.sum(axis = 1) != 0][:, 0]
    return(count_outer_bags, outer_bags)

In [4]:
count_outer_bags, outer_bags = find_outer_bags(look_for=['shiny gold'], bags=bags_map)
print(count_outer_bags)
len(set(outer_bags))

226


226

In [5]:
################################################################
## PART 2
################################################################
def count_bags(outer_bag, multiplier, all_bags):
    inside_bags = all_bags[outer_bag]
    if inside_bags == 'no other bags':
        return(dict())
    amount = np.array(re.findall('\\d', inside_bags))
    amount = amount.astype('int64')
    bags   = re.findall('\\d (.*?) bags?', inside_bags)
    bags_dict = dict(zip(bags, amount * multiplier))

    for bag, n in zip(bags, amount):
        out = count_bags(outer_bag=bag, multiplier=(n*multiplier), all_bags=all_bags)
        for key in out: 
            if key in bags_dict: 
                bags_dict[key] = bags_dict[key] + out[key] 
            else: 
                bags_dict[key] = out[key] 
    return(bags_dict)

In [6]:
bags_map = dict(zip(bags_map[:, 0], bags_map[:, 1]))
gold_bag_inside_bags_map = count_bags(outer_bag='shiny gold', multiplier=1, all_bags=bags_map)
print(sum(gold_bag_inside_bags_map.values()))

9569
